In [1]:
import cobra
import sys
import os

In [2]:
ecpy_path = '../../../ecpy/'
sys.path.append(os.path.abspath(ecpy_path))
import utils
import ecpy

In [3]:
model = cobra.io.load_json_model('../../../ModelFiles/json/Halo_GEM_v1.json')

### Model summary 

In [4]:
def remove_compartment_label(met_id):
    for item in  ['_c','_p','_e']: met_id = met_id.replace(item,'')
    return met_id

In [5]:
def is_transport_rxn(rxn):
    reactants = [remove_compartment_label(met.id) for met in rxn.reactants]
    products = [remove_compartment_label(met.id) for met in rxn.products]
    
    if len(reactants) + len(products) > len(set(reactants+products)): return True
    else: return False

In [6]:
def is_exchange_rxn(rxn):
    if len(rxn.reactants) == 0 or len(rxn.products) == 0: return True
    else: return False

In [7]:
def report_model(model):
    metabolic_rxns = 0
    trans_rxns     = 0
    exchange_rxns  = 0
    
    metabolic_rxns_gr = 0
    trans_rxns_gr     = 0
    
    for rxn in model.reactions:
        gr = 1 if len(rxn.genes)>0 else 0

        if is_transport_rxn(rxn): 
            trans_rxns += 1
            trans_rxns_gr += gr
        elif is_exchange_rxn(rxn): exchange_rxns  += 1
        else: 
            metabolic_rxns += 1
            metabolic_rxns_gr += gr
    
    print('Reactions')
    print('  Metabolic reactions:',metabolic_rxns)
    print('  Transport reactions:',trans_rxns)
    print()
    
    print('Gene assciations')
    print('  Gene associated Metabolic reactions:',metabolic_rxns_gr)
    print('  Gene associated Transport reactions:',trans_rxns_gr)
    print('  No gene associations metabolic reactions:',metabolic_rxns-metabolic_rxns_gr)
    print('  No gene associations transport reactions:',trans_rxns-trans_rxns_gr)
    print()
    
    print('Exchange reactions:',exchange_rxns)
    print()
    
    print('Metabolites')
    print('  Unique metabolites:',len(set([remove_compartment_label(met.id) for met in model.metabolites])))
    print('  Cytoplasmic:',len(set([remove_compartment_label(met.id) for met in model.metabolites if met.compartment=='c'])))
    print('  Periplasmic:',len(set([remove_compartment_label(met.id) for met in model.metabolites if met.compartment=='p'])))
    print('  Extracellular:',len(set([remove_compartment_label(met.id) for met in model.metabolites if met.compartment=='e'])))

In [8]:
report_model(model)

Reactions
  Metabolic reactions: 1863
  Transport reactions: 351

Gene assciations
  Gene associated Metabolic reactions: 1847
  Gene associated Transport reactions: 301
  No gene associations metabolic reactions: 16
  No gene associations transport reactions: 50

Exchange reactions: 56

Metabolites
  Unique metabolites: 1972
  Cytoplasmic: 1912
  Periplasmic: 350
  Extracellular: 77


#### Single gene-knockout

In [9]:
def convert_fronzen_set_to_values(gset):
    ID = tuple([x for x in gset])
    
    if len(ID)==1: return ID[0]
    else: return ID

In [10]:
def modify_model(model):
    # set glucose uptake as 10 mmol/gdw/h
    # set yeast extraction as 1 mmol/gdw/h
    utils.set_bound(model,'Exchange_Glucopyranose',ub=10)
    utils.set_yeast_extraction(model,ub=0,lb=0)
    model.objective = 'Biomass_v1'
    model.objective_direction = 'max'
    print(model.optimize())
    
    model_glc, model_ye = model.copy(), model.copy()
    utils.set_yeast_extraction(model_ye,ub=1,lb=0)
    
    print(model_glc.optimize(),model_ye.optimize())
    return model_glc,model_ye

In [11]:
def save_gene_ko(results,model,outfile):
    new_index = [convert_fronzen_set_to_values(ind) for ind in results.index]
    results.index = new_index
    
    r0 = model.optimize().objective_value
    results['growth'] = results['growth'] / r0
    
    print(results.head())
    results.to_csv(outfile,sep='\t')

In [12]:
model_glc,model_ye = modify_model(model)

<Solution 0.604 at 0x7f4cc40835c0>
<Solution 0.604 at 0x7f4cbe1eb2e8> <Solution 1.381 at 0x7f4cbe1eb550>


In [13]:
single_gene_knockout_glc = cobra.flux_analysis.single_gene_deletion(model_glc)

In [14]:
single_gene_knockout_ye = cobra.flux_analysis.single_gene_deletion(model_ye)

In [15]:
double_gene_knockout_glc = cobra.flux_analysis.double_gene_deletion(model_glc)
double_gene_knockout_ye = cobra.flux_analysis.double_gene_deletion(model_ye)

In [17]:
save_gene_ko(single_gene_knockout_glc,model_glc,'../KO/single_gene_deletion_with_glucose_as_carbon_source.tsv')

              growth   status
TD01GL000690     1.0  optimal
TD01GL003263     1.0  optimal
TD01GL001877     1.0  optimal
TD01GL001911     1.0  optimal
TD01GL001033     1.0  optimal


In [18]:
save_gene_ko(single_gene_knockout_ye,model_ye,'../KO/single_gene_deletion_with_glucose_and_YE_as_carbon_source.tsv')

              growth   status
TD01GL002722     0.0  optimal
TD01GL001885     1.0  optimal
TD01GL001779     1.0  optimal
TD01GL001917     1.0  optimal
TD01GL001753     1.0  optimal


In [19]:
save_gene_ko(double_gene_knockout_glc,model_glc,'../KO/double_gene_deletion_with_glucose_as_carbon_source.tsv')

                                growth   status
(TD01GL003006, TD01GL001466)  1.000000  optimal
(TD01GL003798, TD01GL001170)  1.000000  optimal
(TD01GL001327, TD01GL001562)  0.992142  optimal
(TD01GL002135, TD01GL002764)  1.000000  optimal
(TD01GL000782, TD01GL002160)  0.000000  optimal


In [20]:
save_gene_ko(double_gene_knockout_ye,model_ye,'../KO/double_gene_deletion_with_glucose_and_YE_as_carbon_source.tsv')

                                growth   status
(TD01GL002635, TD01GL000435)  0.931954  optimal
(TD01GL001669, TD01GL002507)  1.000000  optimal
(TD01GL001562, TD01GL003366)  1.000000  optimal
(TD01GL001112, TD01GL002511)  1.000000  optimal
(TD01GL000375, TD01GL002299)  0.000000  optimal


In [29]:
!rm ../KO.tar.gz
!tar -czvf ../KO.tar.gz ../KO

tar: Removing leading `../' from member names
../KO/
../KO/single_gene_deletion_with_glucose_and_YE_as_carbon_source.tsv
../KO/double_gene_deletion_with_glucose_as_carbon_source.tsv
../KO/single_gene_deletion_with_glucose_as_carbon_source.tsv
../KO/double_gene_deletion_with_glucose_and_YE_as_carbon_source.tsv


In [30]:
!du -sh ../KO.tar.gz

14M	../KO.tar.gz
